#### Chapter 9 - Transfer learning

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

In [ ]:
# Load the pre-trained model from TensorFlow
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

Download an image of a shirt and then save the path in the variable image_path

In [ ]:
# Load the image
image_path = "image2.jpeg"
image = load_img(image_path, target_size=(224, 224))

In [ ]:
# convert the image pixels to a numpy array
image = img_to_array(image)
# Preprocess the image
image = preprocess_input(image)

In [ ]:
# Make a prediction using the pre-trained model
prediction = base_model.predict(np.expand_dims(image, axis=0))
# Print the predicted class
predicted_class = np.argmax(prediction)
print("Predicted class:", predicted_class)

In [ ]:
# Getting the predictions labels
decode_predictions(prediction,top=4)

#### Feature extraction

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
# Load the pre-trained model from TensorFlow Hub
base_model_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
base_model = hub.KerasLayer(base_model_url,input_shape=(224, 224, 3),trainable=False)

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# Preprocess the images
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# Add a third dimension of 1 to the train images
x_train_resized = tf.expand_dims(x_train, axis=-1)
x_test_resized = tf.expand_dims(x_test, axis=-1)
print(x_train_resized.shape,x_test_resized.shape)

In [ ]:
# Repeat the channel dimension to get the size as 3
x_train_multi = tf.repeat(x_train_resized, 3, -1)
x_test_multi = tf.repeat(x_test_resized, 3, -1)

print(x_train_multi.shape,x_test_multi.shape)

In [ ]:
# Resize the images to 224,224
x_train = tf.image.resize(x_train_multi, (224, 224))
x_test = tf.image.resize(x_test_multi, (224, 224))

print(x_train.shape,x_test.shape)

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

#### Fine tuning of base model

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train/255
x_test = x_test/255

In [ ]:
# Define the image size
img_size = (128, 128)
# Define the batch size
batch_size = 32
# Define the number of classes in the dataset
num_classes = 10

# Preprocess the images and create a data pipeline
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(buffer_size=len(x_train))
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
train_ds = train_ds.batch(batch_size).prefetch(1)
# Preprocess the test set
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
test_ds = test_ds.batch(batch_size).prefetch(1)

In [ ]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# Freeze the layers in the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add a custom top to the model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

In [ ]:
# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model on the CIFAR-10 dataset with a warmup stage
history = model.fit(train_ds, batch_size=32, epochs=5, validation_data=test_ds)

In [ ]:
# Fine-tune the last few layers of the pre-trained model
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Compile the model again
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model again with fine-tuning
history = model.fit(train_ds, batch_size=32, epochs=10, validation_data=test_ds)

#### Datapipeline for transfer learning

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf


In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Define image size and batch size
img_size = (224, 224)
batch_size = 32

In [ ]:
# Define data augmentation layer for training set
train_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  tf.keras.layers.experimental.preprocessing.Resizing(*img_size),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
])

# Define data augmentation for test layer
test_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(*img_size),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
# Preprocess the images and create a data pipeline with resizing and data augmentation
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(buffer_size=len(x_train))
#train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
train_ds = train_ds.map(lambda x, y: (train_augmentation(x), y))
train_ds = train_ds.batch(batch_size).prefetch(1)

# Preprocess the test set
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (test_augmentation(x), y))
test_ds = test_ds.batch(batch_size).prefetch(1)

In [ ]:
# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(*img_size, 3))


In [ ]:
# Freeze the layers in the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add a custom top to the model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

In [ ]:
# Combine the base model and the custom top
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model on the CIFAR-10 dataset with a warmup stage
history = model.fit(train_ds, batch_size=batch_size, epochs=5, validation_data=test_ds)

In [ ]:
# Unfreeze the last few layers in the pre-trained model for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model on the CIFAR-10 dataset with a warmup stage
history = model.fit(train_ds, batch_size=batch_size, epochs=10, validation_data=test_ds)

#### Transfer learning for NLP tasks

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[:70%]', 'train[70%:]', 'test'),as_supervised=True)


In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
embedding_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
embedding_layer(train_examples_batch[:3])

In [ ]:
model = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(256),
                    epochs=5,
                    validation_data=validation_data.batch(256),
                    verbose=1)